# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [15]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key



## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [16]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_data_to_load= "C:/Users/casey/OneDrive/Desktop/Course Work/World Weather Analysis/Vacation_Search/WeatherPy_vacation.csv"
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(10)

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,0,Micheweni,TZ,-4.9667,39.8333,78.15,84,25,8.37,light rain,2022-11-28 23:11:57
1,1,1,Sorong,ID,-0.8833,131.2500,80.04,82,100,0.31,overcast clouds,2022-11-28 23:11:58
2,2,2,Kaitangata,NZ,-46.2817,169.8464,58.84,68,100,6.38,overcast clouds,2022-11-28 23:11:59
3,3,3,Ushuaia,AR,-54.8000,-68.3000,62.26,34,40,12.66,scattered clouds,2022-11-28 23:11:35
4,4,4,Hilo,US,19.7297,-155.0900,80.62,94,100,11.99,light rain,2022-11-28 23:11:59
5,5,5,Fortuna,US,40.5982,-124.1573,51.66,79,87,5.99,overcast clouds,2022-11-28 23:08:44
6,6,6,Coquimbo,CL,-29.9533,-71.3436,60.53,88,75,9.22,broken clouds,2022-11-28 23:11:59
7,7,7,Verkhniy Landekh,RU,56.8389,42.5994,8.76,94,76,4.27,broken clouds,2022-11-28 23:12:00
8,8,8,Nikolskoye,RU,59.7035,30.7861,26.49,96,93,4.50,overcast clouds,2022-11-28 23:12:00
9,9,9,Tuktoyaktuk,CA,69.4541,-133.0374,-11.29,99,79,9.84,broken clouds,2022-11-28 23:12:01


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [17]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
     "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name","Country", "Current Description"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [18]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[ vacation_df["City"]=="Flagstaff"]
vacation_end = vacation_df.loc[ vacation_df["City"]=="Flagstaff"]
vacation_stop1 = vacation_df.loc[ vacation_df["City"]=="Chico"]
vacation_stop2 = vacation_df.loc[ vacation_df["City"]=="Altamont"]
vacation_stop3 = vacation_df.loc[ vacation_df["City"]=="Rancho Palos Verdes"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [19]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = itinerary_df =pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3,vacation_end])

# Display sample data
itinerary_df.head()

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
684,684,684,Flagstaff,US,35.1981,-111.6513,46.53,36,0,26.46,clear sky,2022-11-28 23:29:22
682,682,682,Chico,US,39.7285,-121.8375,61.21,37,40,5.75,scattered clouds,2022-11-28 23:28:02
640,640,640,Altamont,US,42.2068,-121.7372,34.39,83,75,10.36,broken clouds,2022-11-28 23:29:41
293,293,293,Rancho Palos Verdes,US,33.7445,-118.3870,63.99,76,75,8.05,broken clouds,2022-11-28 23:19:54
684,684,684,Flagstaff,US,35.1981,-111.6513,46.53,36,0,26.46,clear sky,2022-11-28 23:29:22


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [20]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lat','Lng']]

# Display sample data
waypoints_df.head()

,Lat,Lng
684,35.1981,-111.6513
682,39.7285,-121.8375
640,42.2068,-121.7372
293,33.7445,-118.3870
684,35.1981,-111.6513


## Use GeoViews to create map that shows the four cities in the itinerary

In [22]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
     "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name","Country", "Current Description"]
)

In [23]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [24]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [25]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'ca736fb05110436691dbc3824b27cbaf',
 'waypoints': '35.1981,-111.6513|39.7285,-121.8375|42.2068,-121.7372|33.7445,-118.387|35.1981,-111.6513'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [26]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [27]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [28]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [29]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df.head()

,longitude,latitude
0,-111.651261,35.198186
1,-111.651398,35.198228
2,-111.652445,35.198567
3,-111.652726,35.198658
4,-111.652871,35.198692


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [30]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "red"
)

In [31]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]